# Movies 3

## Introduction

Une base de données (hébergée dans le cloud AWS) contenant un certain nombre de données cinématographiques vous est fournie pour exploration et mise en pratique des langages SQL et Python.  
Les données de connexion (user, host) sont indiquées dans le brief, le mot de passe nécessaire est précisé en session par le formateur.  


## 1. Description Générale

En utilisant DBeaver et les paramètres de connexion donnés par ailleurs, faire une description détaillée de la base de données fournie. Présenter notamment le diagramme entité - relation (ERD) et les différentes caractéristiques des tables de la base.  Quelles types de renseignement peut-on tirer de cette base de données ?   

Nous disposons de 4 tables: 

1) movies:


`id`:integer définit comme Primary Key (int)

`title`:varchar(80)

`year`:integer (int)

`directorID`: integer définit comme foreign key (table reférenece artists id)

`genre`: varchar(20)

`summary`:Text

`countrycode`:Varchar(4) définit comme foreign key (table reférence countries code)

2) roles:

`movieId`:integer définit comme Primary Key et aussi Foreign Key (table reférence movies id) (int)

`actorId`:integer définit comme Primary Key et aussi Foreign Key (table reférence artists id) (int)

`character`:varchar(255)

3) artists:



`id`:integer définit comme Primary Key (reférence à la colonne actorId de la table roles et directorId table movies) (int)

`lastname`:varchar(30)

`firstname`:varchar(30)

`birthyear`:integer(int)

4) countries:

`code`:varchar(4) définit comme Primary Key (reférenece à la colonne countrycode de la table movies)

`name`:varchar(30)

`language`:varchar(30)

---

## 2. Exploration SQL via python / pandas
Dans cette partie, on s'attache à explorer la base de données en utilisant le language SQL pour les requêtes et le language Python (+ module pandas) pour récupérer les résultats de façon programmatique.  
Vous pouvez utiliser DBeaver ou outil similaire pour préparer / tester vos requêtes mais celles-ci doivent être reportées et appliquées dans du code Python comme présenté dans les quelques examples à suivre.  
*Pour les résultats "longs", un aperçu des quelques premières lignes et le comptage total des résultats est suffisant.*

In [ ]:
import os
import pandas as pd

In [ ]:
from sqlalchemy import create_engine
import pymysql

In [ ]:
# paramètres de connexion à la base de données movies (MySQL sous AWS RDS)
user = os.environ.get('movies3_user')
password = os.environ.get('movies3_psw') 
host = os.environ.get('movies3_host')  
port = 3306
dbname = 'movies'  

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}")

### Exemples

In [ ]:
print(user)

#### Tables de base

In [ ]:
movies = pd.read_sql_query("SELECT * FROM movies LIMIT 10", engine)
movies.head()

In [ ]:
artists = pd.read_sql_query("SELECT * FROM artists LIMIT 10", engine)
artists.head()

In [ ]:
roles = pd.read_sql_query("SELECT * FROM roles LIMIT 10", engine)
roles.head()

In [ ]:
countries = pd.read_sql_query("SELECT * FROM countries LIMIT 10", engine)
countries.head()

#### Autre requête simple
Q00: Nom et année de naissance des artistes nés après 1975 ? Combien sont-ils ?

In [ ]:
sql_query_00 = """
    SELECT *
    FROM artists
    WHERE birthyear > 1975
"""

results = pd.read_sql_query(sql_query_00, engine)
results.head(5)

In [ ]:
results.shape

In [ ]:
len(results)

### Questions

Q01: Quels sont les films de type 'Drame' ? Combien cela représente de films ?  

In [ ]:
sql_query_01 = """
    SELECT genre as "genre", title as "titre du film", count(tilte)
    FROM movies
    WHERE genre ='Drame'
"""

results = pd.read_sql_query(sql_query_01, engine)
results.head()

In [ ]:
len(results)

Q02: Quels rôles a joué Bruce Willis et dans quels films (triés par ordre chronologique) ?

In [ ]:
sql_query_02 = """
    SELECT title AS "Titre", year AS "Année", roles.character AS "Rôle", lastname AS "Nom", firstname AS "Prénom"
    FROM movies
    INNER JOIN roles ON movies.id = roles.movieId
    INNER JOIN artists ON roles.actorId = artists.id
    WHERE lastname = "Willis" AND firstname = "Bruce"
    ORDER BY year;
"""

results = pd.read_sql_query(sql_query_02, engine)
results.head(5)

Q03: Qui est le réalisateur de Memento ?

In [ ]:
sql_query_03 = """
    SELECT movies.title as "Titre du film", artists.id as "ID artiste", artists.lastname as "Nom", artists.firstname as "Prénom"
    FROM artists
    LEFT JOIN movies ON artists.id = movies.directorId
    WHERE artists.id = 525 AND movies.title = "Memento";
"""

results = pd.read_sql_query(sql_query_03, engine)
results.head(5)

Q04: Qui a joué le rôle de Chewbacca et dans quels films ?

In [ ]:
sql_query_04 = """
    SELECT title AS "Titre", roles.character AS "Rôle", lastname AS "Nom", firstname AS "Prénom"
    FROM roles
    INNER JOIN artists ON roles.actorId = artists.id
    INNER JOIN movies ON roles.movieId = movies.id
    WHERE roles.character = "Chewbacca";
"""

results = pd.read_sql_query(sql_query_04, engine)
results.head()

Q05: Dans quels films Bruce Willis a-t-il joué le role de John McClane ? 

In [ ]:
sql_query_05 = """
    SELECT title as "Titre des films", lastname as "Nom", firstname as "Prénoms", roles.character as "Nom du personnage"
    FROM movies
    INNER JOIN roles ON movies.id = roles.movieId
    INNER JOIN artists ON roles.actorId = artists.id
    WHERE lastname = "Willis" AND firstname = "Bruce" AND     
    roles.character = "John McClane"
"""

results = pd.read_sql_query(sql_query_05, engine)
results.head(5)

Q06: Quels sont les acteurs de 'Sueurs froides' ?

In [ ]:
sql_query_06 = """
    SELECT title AS "Titre", lastname AS "Nom", firstname AS "Prénom", roles.character AS "Rôle"
    FROM movies
    INNER JOIN roles ON movies.id = roles.movieId
    INNER JOIN artists ON roles.actorId = artists.id
    WHERE title = "Sueurs froides";
"""

results = pd.read_sql_query(sql_query_06, engine)
results.head()

Q07: Quels sont les films dont le réalisateur est Tim Burton, et l’un des acteurs Jonnhy Depp ?

In [ ]:
sql_query_07 = """
    SELECT title as "Titre du film"
        FROM movies
        INNER JOIN artists ON movies.directorId = artists.id
        WHERE lastname = "Burton"
    UNION
    SELECT title
        FROM movies
        INNER JOIN roles ON movies.id = roles.movieId
        INNER JOIN artists ON roles.actorId = artists.id
        WHERE lastname = "Depp"
"""

results = pd.read_sql_query(sql_query_07, engine)
results.head()

Q08: Quels sont les films dans lesquels a joué Woody Allen ? Donnez aussi le rôle.

In [ ]:
sql_query_08 = """
    SELECT title AS "Titre", lastname AS "Nom", firstname AS "Prénom", roles.character AS "Rôle"
    FROM movies
    INNER JOIN roles ON movies.id = roles.movieId
    INNER JOIN artists ON roles.actorId = artists.id
    WHERE lastname = "Allen" AND firstname = "Woody"
"""

results = pd.read_sql_query(sql_query_08, engine)
results.head()

Q09: Quel metteur en scène a tourné dans ses propres films ? Donnez le nom, le rôle et le titre des films en question.

In [ ]:
sql_query_09 = """
    SELECT lastname AS "Nom", firstname AS "Prénom", title AS "Titre", roles.character AS "Rôle"
    FROM movies
    INNER JOIN roles ON movies.id = roles.movieId
    INNER JOIN artists ON roles.actorId = artists.id
    WHERE movies.directorId = roles.actorId;
"""

results = pd.read_sql_query(sql_query_09, engine)
results.head()

Q10: Quels sont les films de Quentin Tarantino dans lesquels il n’a pas joué 

In [ ]:
sql_query_10 = """
    SELECT a.title as "Titre des films"
      FROM movies AS a
      INNER JOIN artists ON a.directorId = artists.id
      WHERE lastname = "Tarantino"
      AND title NOT IN (
        SELECT b.title 
        FROM movies AS b
        INNER JOIN roles ON b.id = roles.movieId
        INNER JOIN artists ON roles.actorId = artists.id
        WHERE lastname = "Tarantino")
"""

results = pd.read_sql_query(sql_query_10, engine)
results.head()

Q11: Quel metteur en scène a tourné en tant qu’acteur ? Donner le nom, le rôle et le titre des films dans lesquels cet artiste a joué.

In [ ]:
sql_query_11 = """
    SELECT lastname as "Nom", roles.character as "Role", title as "Titre du film", roles.actorId as "ID Acteur", movies.directorId as "ID Metteur en scène" 
    FROM movies
    INNER JOIN artists ON movies.directorId = artists.id
    INNER JOIN roles ON movies.id = roles.movieId
    WHERE movies.directorId = roles.actorId;
"""

results = pd.read_sql_query(sql_query_11, engine)
results.head()

Q12: Quels sont les films de Hitchcock sans James Stewart

In [ ]:
sql_query_12 = """
    SELECT a.title as "Titre des films"
    FROM movies AS a
    INNER JOIN artists ON a.directorId = artists.id
    WHERE lastname = "Hitchcock"
    AND title NOT IN (
      SELECT b.title 
      FROM movies AS b
      INNER JOIN roles ON b.id = roles.movieId
      INNER JOIN artists ON roles.actorId = artists.id
      WHERE lastname = "Stewart")
"""

results = pd.read_sql_query(sql_query_12, engine)
results.head()

Q13: Quel est la filmographie de James Stewart ?

In [ ]:
sql_query_13 = """
    SELECT lastname as "Nom", firstname as "Prénom", title AS "Titre"
    FROM movies
    INNER JOIN roles ON movies.id = roles.movieId
    INNER JOIN artists ON roles.actorId = artists.id
    WHERE lastname = "Stewart" AND firstname = "James";
"""

results = pd.read_sql_query(sql_query_13, engine)
results.head()

Q14: Quels sont les films sans rôle (acteurs) ?

Q15: Combien d'acteurs n’ont jamais réalisé de film ? 

In [ ]:
sql_query_15 = """
    SELECT DISTINCT artists.id AS "id", lastname as "Nom", firstname AS "Prénom"
    FROM movies
    INNER JOIN roles ON movies.id = roles.movieId
    INNER JOIN artists ON roles.actorId = artists.id
    WHERE artists.id NOT IN (
      SELECT DISTINCT artists.id
      FROM movies
      INNER JOIN artists ON movies.directorId = artists.id)
    ORDER BY artists.id;
"""

results = pd.read_sql_query(sql_query_15, engine)
results.head()

Q16: Donnez les noms et prénoms des 10 réalisateurs les plus prolifiques, ainsi que le nombre de films qu’ils ont tournés ?

Q17: Donnez les noms et prénoms des réalisateurs qui ont tourné au moins deux films. Combien sont-ils ?

Q18: Dans quels films le réalisateur a-t-il le même prénom que l’un des interprètes ? (titre, nom du réalisateur, nom de l’interprète). Le réalisateur et l’interprète ne doivent pas être la même personne.

Q19: Quels sont les 10 pays produisant le plus de films ?

Q20: Quels sont les 5 acteurs ayant le plus joué dans des films français ?

---

## 3. Expression libre

A vous de jouer ! Tentez quelques requêtes de votre choix (avec pourquoi pas quelques graphiques illustratifs en utilisant Matplotlib...)